In [1]:
import numpy as np 
import pandas as pd 

np.random.seed(100)

In [46]:
def Create_Warehouses(NumberOfWarehouses=5, Latitude_Range=[34, 35], Longitude_Range=[-81, -80]):

    Warehouse_df = pd.DataFrame({
        "WarehouseID": pd.Series(dtype=int),
        "WH_Latitude": pd.Series(dtype=float),
        "WH_Longitude": pd.Series(dtype=float),
        "Volume": pd.Series(dtype=int)
    })

    Warehouse_df["WarehouseID"] = np.arange(1, NumberOfWarehouses + 1, 1, dtype=int) * 1000
    Latitude_Min, Latitude_Max = Latitude_Range[0], Latitude_Range[1]
    Longitude_Min, Longitude_Max = Longitude_Range[0], Longitude_Range[1]

    Warehouse_df["WH_Latitude"] = np.random.uniform(Latitude_Min, Latitude_Max, NumberOfWarehouses)
    Warehouse_df["WH_Longitude"] = np.random.uniform(Longitude_Min, Longitude_Max, NumberOfWarehouses)

    Warehouse_df["Volume"] = np.random.randint(2, 8, NumberOfWarehouses) # The volume each WH gets

    return Warehouse_df

wh_df = Create_Warehouses(NumberOfWarehouses=10)
wh_df

,WarehouseID,WH_Latitude,WH_Longitude,Volume
0,1000,34.971992,-80.574859,7
1,2000,34.121397,-80.844319,4
2,3000,34.465521,-80.946181,2
3,4000,34.277520,-80.597665,3
4,5000,34.328124,-80.619202,6
5,6000,34.031746,-80.037795,3
6,7000,34.467139,-80.012999,5
7,8000,34.536336,-80.825164,2
8,9000,34.053818,-80.333497,3
9,10000,34.015472,-80.492392,3


In [47]:
iter_df = pd.DataFrame({"ShipmentDate": pd.date_range(start="2025-01-01", end="2025-08-31").tolist()})
df = pd.DataFrame(columns=["ShipmentID", "ShipmentDate", "WarehouseID", "ItemCount", "TotalWeight", "COGS", "Revenue", "Profit", "ShipmentType"])

for i in range(len(iter_df)):
    ShipmentDate = pd.to_datetime(iter_df.ShipmentDate.iloc[i])

    shipment_types = ["standard", "standard", "standard", "expedited", "expedited", "express", "express", "express", "overnight"]
    shipment_types = [s_type.upper() for s_type in shipment_types]


    for j in range(len(wh_df)):

        join_df = pd.DataFrame()

        WarehouseID = wh_df.WarehouseID.iloc[j]
        Volume = wh_df.Volume.iloc[j]

        cnt_shipments = np.random.randint(low=0, high=Volume)

        join_df["ShipmentID"] = np.random.randint(10000, 99999, size=cnt_shipments)
        join_df["ShipmentDate"] = [ShipmentDate for i in range(cnt_shipments)]
        join_df["WarehouseID"] = [WarehouseID for i in range(cnt_shipments)]
        join_df["ItemCount"] = np.random.randint(100, 1000, size=cnt_shipments)
        join_df["TotalWeight"] = np.random.uniform(1000, 10000, size=cnt_shipments)
        join_df["COGS"] = join_df["TotalWeight"] * np.random.uniform(0, 0.5, size=cnt_shipments)
        join_df["Revenue"] = join_df["TotalWeight"] * np.random.uniform(2, 3, size=cnt_shipments)
        join_df["Profit"] = join_df["Revenue"] - join_df["COGS"]
        join_df["ShipmentType"] = np.random.choice(shipment_types, size=cnt_shipments)

        df = pd.concat([df, join_df])


In [48]:
df = pd.merge(df, wh_df, on="WarehouseID", suffixes=('', '_y'))
df = df.drop("Volume", axis=1)
df

,ShipmentID,ShipmentDate,WarehouseID,ItemCount,TotalWeight,COGS,Revenue,Profit,ShipmentType,WH_Latitude,WH_Longitude
0,16730,2025-01-01 00:00:00,1000,409,4420.680653,538.678497,8970.589513,8431.911017,OVERNIGHT,34.971992,-80.574859
1,30890,2025-01-01 00:00:00,4000,141,6079.425418,1943.304356,12970.391644,11027.087287,EXPRESS,34.277520,-80.597665
2,35247,2025-01-01 00:00:00,5000,988,5388.235547,173.352133,16095.555989,15922.203856,EXPEDITED,34.328124,-80.619202
3,77127,2025-01-01 00:00:00,7000,433,7259.925285,2730.704661,16369.279127,13638.574466,STANDARD,34.467139,-80.012999
4,69636,2025-01-01 00:00:00,7000,406,3190.417191,859.43007,9061.190704,8201.760633,EXPEDITED,34.467139,-80.012999
...,...,...,...,...,...,...,...,...,...,...,...
3415,15288,2025-08-31 00:00:00,5000,837,6274.973963,1291.576634,13969.264873,12677.688239,EXPRESS,34.328124,-80.619202
3416,63853,2025-08-31 00:00:00,5000,192,9781.604459,3147.292094,24390.1679,21242.875806,EXPRESS,34.328124,-80.619202
3417,78500,2025-08-31 00:00:00,5000,341,6843.674524,3151.022085,16679.148634,13528.126549,EXPRESS,34.328124,-80.619202
3418,38064,2025-08-31 00:00:00,10000,911,7410.9028,1245.311609,19452.521882,18207.210272,EXPRESS,34.015472,-80.492392


In [43]:
df.to_excel("Warehouse_Data.xlsx")

In [44]:
group_df = df.copy()

group_df["ShipmentMonth"] = pd.to_datetime(group_df.ShipmentDate).dt.month
group_df["ShipmentMonthName"] = pd.to_datetime(group_df.ShipmentDate).dt.month_name().str[:3]
group_df = group_df.groupby(["ShipmentMonth", "ShipmentMonthName"]).agg({
    "ShipmentID": "count",
    "TotalWeight": "sum",
    "COGS": "sum",
    "Revenue": "sum",
    "Profit": "sum"
})

group_df_shift = group_df.shift(1)

group_df = group_df.reset_index()
group_df_shift = group_df_shift.reset_index()

goals_df = pd.merge(group_df, group_df_shift, on="ShipmentMonth", suffixes=("", "_Goal"))
goals_df = goals_df.drop("ShipmentMonthName_Goal", axis=1)
goals_df

,ShipmentMonth,ShipmentMonthName,ShipmentID,TotalWeight,COGS,Revenue,Profit,ShipmentID_Goal,TotalWeight_Goal,COGS_Goal,Revenue_Goal,Profit_Goal
0,1,Jan,507,2792837.165013,701908.885114,6982556.483854,6280647.598739,NaN,None,None,None,None
1,2,Feb,462,2601291.318499,637058.863623,6482043.934666,5844985.071044,507.0,2792837.165013,701908.885114,6982556.483854,6280647.598739
2,3,Mar,563,3011833.804045,749904.62161,7468064.018869,6718159.397259,462.0,2601291.318499,637058.863623,6482043.934666,5844985.071044
3,4,Apr,486,2655861.341491,652425.13129,6631559.916667,5979134.785377,563.0,3011833.804045,749904.62161,7468064.018869,6718159.397259
4,5,May,496,2665165.693381,701789.758047,6691733.050147,5989943.2921,486.0,2655861.341491,652425.13129,6631559.916667,5979134.785377
5,6,Jun,473,2647079.394493,667569.97752,6615602.931952,5948032.954432,496.0,2665165.693381,701789.758047,6691733.050147,5989943.2921
6,7,Jul,469,2647074.859254,641390.423588,6667248.58809,6025858.164503,473.0,2647079.394493,667569.97752,6615602.931952,5948032.954432
7,8,Aug,519,2841693.401474,684453.622366,7079406.076932,6394952.454566,469.0,2647074.859254,641390.423588,6667248.58809,6025858.164503


In [45]:
goals_df.to_excel("Warehouse_Data_Goals.xlsx")